# Example - Opportunity challenge.


#### Download and preprocess data

First, download the dataset for the opportunity challenge and set up the directories to hold the raw and processed data:

In [1]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
! mkdir -p data/raw
! mv OpportunityUCIDataset.zip data/raw

--2021-09-29 12:50:26--  https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306636009 (292M) [application/x-httpd-php]
Saving to: ‘OpportunityUCIDataset.zip’

OpportunityUCIDatas 100%[===================>] 292.43M  8.53MB/s    in 66s     

2021-09-29 12:51:33 (4.44 MB/s) - ‘OpportunityUCIDataset.zip’ saved [306636009/306636009]



Then, run the following script to preprocess the data. We collate all the training data into one file, same for the validation and testing data.

In [2]:
! python3 preprocess_opportunity.py

Checking dataset data/raw/OpportunityUCIDataset.zip
Processing dataset files ...
Generating training files
... file OpportunityUCIDataset/dataset/S1-Drill.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL1.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL2.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL3.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL4.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-Drill.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-ADL1.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-ADL2.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-Drill.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-ADL1.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-ADL2.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-ADL3.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-ADL3.dat -> train_data
Generating validation files
... file Opportun

#### Dataset Config

We choose the parameters of our sliding window, and specify the name and location of the target dataset.

In [1]:
target_dataset = 'opportunity'
window_size = 24
window_step = 12
n_classes = 18

config_dataset = {
        "dataset": target_dataset,
        "window": window_size,
        "stride": window_step,
        "stride_test": 1,
        "path_processed": f"data/{target_dataset}",
    }

Import the data for training and validation (see datasets.py for sensor dataset implementation). We get the number of channels from the dataset.

In [2]:
from datasets import SensorDataset
dataset = SensorDataset(**config_dataset, prefix="train")
dataset_val = SensorDataset(**config_dataset, prefix="val")
n_channels = dataset.n_channels


Creating opportunity train HAR dataset of size 43985 ...
Creating opportunity val HAR dataset of size 2509 ...


### Import DeepConvLSTM class

See DeepConvLSTM_py3.py for implementation.

In [3]:
from DeepConvLSTM_py3 import DeepConvLSTM

#### Create an instance of DeepConvLSTM, with the number of channels and classes defined earlier. 
The dataset arg determines where the results and training checkpoints should be saved.


In [4]:
deepconv = DeepConvLSTM(n_channels=n_channels, n_classes=n_classes, dataset=target_dataset).cuda()

### Train the model


We'll train for only 30 epochs here as a demonstration. We use a learning rate scheduler to decrease the maximum learning rate for all parameters every 10 epochs, by a factor of 0.9. Check model_train docstring for explanation of the config keys.

In [5]:
from DeepConvLSTM_py3 import model_train

# Define train config options
config_train = {'batch_size': 256,
                'optimizer': 'Adam',
                'lr': 1e-3,
                'lr_step': 10,
                'lr_decay': 0.9,
                'init_weights': 'orthogonal',
                'epochs': 30,
                'print_freq': 100
               }

model_train(deepconv, dataset, dataset_val, config_train, verbose=True)

Running HAR training loop ...
[-] Initializing weights (orthogonal)...
----------------------------------------------------------------------------------------------------
[-] Learning rate:  0.001
[-] Batch 0/172	 Loss: 2.895446
[-] Batch 100/172	 Loss: 1.408444
[-] Epoch 0/30	Train loss: 1.00 	acc: 73.26(%)	fm: 10.31(%)	fw: 64.02(%)	
[-] Epoch 0/30	Val loss: 0.62 	acc: 83.38(%)	fm: 9.96(%)	fw: 78.61(%)
[*] Saving checkpoint... (0.0->0.09962714055342088)
----------------------------------------------------------------------------------------------------
[-] Learning rate:  0.001
[-] Batch 0/172	 Loss: 1.199953
[-] Batch 100/172	 Loss: 0.913685
[-] Epoch 1/30	Train loss: 0.75 	acc: 77.66(%)	fm: 22.82(%)	fw: 72.01(%)	
[-] Epoch 1/30	Val loss: 0.47 	acc: 87.41(%)	fm: 21.40(%)	fw: 83.77(%)
[*] Saving checkpoint... (0.09962714055342088->0.2139574317938737)
----------------------------------------------------------------------------------------------------
[-] Learning rate:  0.001
[-] Batc

Load the testing data to evaluate the trained model, and setup the test configuration.

In [6]:
dataset_test = SensorDataset(**config_dataset, prefix="test")
test_config = {'batch_size': 256,
              'train_mode': False,
              'dataset': target_dataset,
              'num_batches_eval': 212}

Creating opportunity test HAR dataset of size 118726 ...


Import model_eval and test the model.

In [7]:
from DeepConvLSTM_py3 import model_eval
model_eval(deepconv, dataset_test, test_config, return_results=False)

Running HAR evaluation loop ...
[-] Loading checkpoint ...
[-] Test loss: 0.52	acc: 89.20(%)	fm: 58.28(%)	fw: 89.00(%)
[Finished HAR evaluation loop (h:m:s): 0:01:18


We can also use [thop](https://pypi.org/project/thop/) to count the number of floating point operations performed during a forward pass of one batch of synthetic data. We divide by the batch size to get the number of operations per window.

In [14]:
# Get number of flops
from thop import profile
import torch
deepconv = deepconv.train()

x = torch.ones([config_train['batch_size'], config_dataset['window'], n_channels]).cuda()
macs, params = profile(deepconv, inputs=(x,), verbose=True)
flops = macs / config_train['batch_size']
print(f'Number of floating point operations per forward pass of one sliding window segment: {flops:,}')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_lstm() for <class 'torch.nn.modules.rnn.LSTM'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'DeepConvLSTM_py3.DeepConvLSTM'>. Treat it as zero Macs and zero Params.
Number of floating point operations per forward pass of one sliding window segment: 115,671,040.0
